# Creación de Modelo de Machine Learning

Este notebook realizará la creación y entrenamiento de un modelo de machine learning.

## Pasos a implementar:

- Cargar librerías de procesamiento y visualización de datos
- Cargar el CSV con las coordenadas rastreadas por SLEAP
- Utilizar librerías para análisis estadístico y preprocesamiento
- Entrenar y evaluar el modelo

In [ ]:
# TODO: Implementar este notebook